In [2]:
import pandas as pd
import numpy as np
import numpy as np
import matplotlib.pylab as plt
import math
import random

In [4]:
df=pd.read_csv('louisville.csv')
df.insert(df.shape[1],'group attribute',0)
df.loc[df['service_name']== 'SMOKE DETECTOR' ,'group attribute']=1
df.loc[df['service_name']== 'VAP EXTREIOR' ,'group attribute']=1
df.loc[df['service_name']== 'PESTS' ,'group attribute']=1
df.loc[df['service_name']== 'MARKING NEW' ,'group attribute']=1
df.loc[df['service_name']== 'PARK MAINTAIN' ,'group attribute']=1
df.loc[df['service_name']== 'MISCELLANEOUS' ,'group attribute']=1

In [5]:
df['address']=df['address'].str.slice(-5,)
df['requested_datetime']=df['requested_datetime'].str.slice(0,11)

In [7]:
df['requested_datetime']= pd.to_datetime(df['requested_datetime'])
df5=df.loc[:,['requested_datetime','address','group attribute']]

In [8]:
group_obj = df5.groupby(['address','group attribute'])

In [14]:
list1=df5.address.tolist()
list2=random.sample(list1,10)

In [13]:
list_1=[]


In [11]:
list3,list4,list5=[],[],[]
list5=np.random.randint(10,150,20).tolist()
len(list5)

20

In [16]:
## ground truth data
for i in list2:
    a1 = group_obj.get_group((i,1))
    a2 = group_obj.get_group((i,0))
    b1=a1.groupby('requested_datetime').size()
    b2=a2.groupby('requested_datetime').size()
    
    c1 = pd.DataFrame(b1)
    c1.reset_index(inplace=True)
    c2 = pd.DataFrame(b2)
    c2.reset_index(inplace=True)
    
    d1=c1.rename(columns={0:'gp'})
    d2=c2.rename(columns={0:'gp'})
    e1=d1['gp'].tolist()
    e2=d2['gp'].tolist()
    list_1.append(e1)
    list_2.append(e2)

In [59]:
### partial ground truth data
list_p0=[] 
list_p1=[]
for i in list2:
    a = group_obj.get_group((i,1))
    A = group_obj.get_group((i,0))
    b=a.groupby('requested_datetime').size()
    B=A.groupby('requested_datetime').size()
    
    c = pd.DataFrame(b)
    c.reset_index(inplace=True)
    C = pd.DataFrame(B)
    C.reset_index(inplace=True)
    
    d=c.rename(columns={0:'gp'})
    D=C.rename(columns={0:'gp'})
    f=d.sample(frac=0.1).index.tolist()
    F=D.sample(frac=0.1).index.tolist()
    for j in f:
        d['gp'][j]=np.random.randint(2,10)
    for g in F:
        D['gp'][g]=np.random.randint(10,30)
    
    list_p0.append(d['gp'].tolist())
    list_p1.append(D['gp'].tolist())

<ipython-input-59-99bf50f581cc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['gp'][j]=np.random.randint(2,10)
<ipython-input-59-99bf50f581cc>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  D['gp'][g]=np.random.randint(10,30)
<ipython-input-59-99bf50f581cc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['gp'][j]=np.random.randint(2,10)
<ipython-input-59-99bf50f581cc>:22: SettingWithCopyWarning: 
A value is trying to be se

In [18]:
### randomly lost value
list_r=[]
list_R=[]
for i in list2:
    a_1 = group_obj.get_group((i,1))
    A_1 = group_obj.get_group((i,0))
    
    b_1=a_1.groupby('requested_datetime').size()
    B_1=A_1.groupby('requested_datetime').size()
    c_1 = pd.DataFrame(b_1)
    C_1=pd.DataFrame(B_1)
    c_1.reset_index(inplace=True)
    C_1.reset_index(inplace=True)
    
    d_1=c_1.rename(columns={0:'gp'})
    D_1=C_1.rename(columns={0:'gp'})
    for j in list5:
        d_1['gp'][j]=np.nan
        D_1['gp'][j]=np.nan
    list_r.append(d_1)
    list_R.append(D_1)

<ipython-input-18-8174aca5b4ce>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_1['gp'][j]=np.nan
<ipython-input-18-8174aca5b4ce>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  D_1['gp'][j]=np.nan
<ipython-input-18-8174aca5b4ce>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_1['gp'][j]=np.nan
<ipython-input-18-8174aca5b4ce>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See t

In [19]:
### imputation model
list_imputation=[]
import scipy
from scipy.interpolate import interp1d
for y in list_r:
    y['requested_datetime'] = np.arange(y.shape[0])
    df_nona = y.dropna(subset=['gp'])
    f = interp1d(df_nona['requested_datetime'], df_nona['gp'],fill_value="extrapolate")
    ip=f(y['requested_datetime'])
    list_imputation.append(ip)  

In [22]:
list_imputation1=[]
for i in list_R:
    i['requested_datetime'] = np.arange(i.shape[0])
    df_nona1 = i.dropna(subset=['gp'])
    g = interp1d(df_nona1['requested_datetime'], df_nona1['gp'])
    ip1=g(i['requested_datetime']).tolist()
    list_imputation1.append(ip1) 

In [23]:
### calculate the estimator
q=0
for i in list_imputation:
    h=sum(i)
    q=q+h
q

2461.5

In [24]:
q1=0
for i in list_imputation1:
    h1=sum(i)
    q1=q1+h1
q1

58286.0

In [25]:
q2=0
for i in list_1:
    h2=sum(i)
    q2=q2+h2
q2

2455

In [26]:
q3=0
for i in list_2:
    h3=sum(i)
    q3=q3+h3
q3

58567

In [60]:
q4=0
for i in list_p0:
    h4=sum(i)
    q4=q4+h4
q4

3022

In [61]:
q5=0
for i in list_p1:
    h5=sum(i)
    q5=q5+h5
q5

59068

In [62]:
estimator=q3-q1+q2-q4+q5-q3+q4-q2
estimator

782.0

In [40]:
## lowerbound
listv=[]
list_v=[]
for i in list2:
    a3 = group_obj.get_group((i,1))
    A3 = group_obj.get_group((i,0))
    
    b3=a3.groupby('requested_datetime').size()
    B3=A3.groupby('requested_datetime').size()
    c3 = pd.DataFrame(b3)
    C3=pd.DataFrame(B3)
    c3.reset_index(inplace=True)
    C3.reset_index(inplace=True)
    
    d3=c3.rename(columns={0:'gp'})
    D3=C3.rename(columns={0:'gp'})
    listv=listv+d3.gp.tolist()
    list_v=list_v+(D3.gp.tolist())
lower_bound=(math.sqrt(np.var(listv))+math.sqrt(np.var(list_v)))/24
lower_bound

0.5917499228809344

In [48]:
### upperbound
### total difference p=1
diff=max(max(listv),max(list_v))-min(min(listv),min(list_v))
### total variation difference:
vdiff=(max(listv)-min(listv))+(max(list_v)-min(list_v))
## expectation
expectation=q2+q3-q-q1
## sum Aa : here set delta = 0.1, c=expectation
v1=math.sqrt(np.var(listv))
v2=math.sqrt(np.var(list_v))
v=(v1+v2)*math.sqrt(math.log(10)*2)
rest=math.log(10)*2*(q2+q3-q-q1)/(3*len(df5))
sumAa=rest+v
upper_bound=sumAa+expectation+diff*vdiff

In [49]:
upper_bound

4984.981131181617